In [8]:
import pandas as pd
import numpy as np
import re, pyperclip,codecs
from IPython.core.display import display, HTML

In [9]:
repChaines="/Users/gilles/pCloud Drive/FOD/GB/Copy/Recherche/Rugby/ChainesReferences/"
fChaines=repChaines+"FR-AR-fr-GB.xlsx"
# fChaines=repChaines+"FR-NZ-en-GB.xlsx"
fChainesCorr=fChaines.replace("-GB.xlsx","-GB-corr.xlsx")
print fChainesCorr
fChainesDiff=fChaines.replace("-GB.xlsx","-GB-diff.xlsx")
print fChainesDiff
fChainesExport=fChaines.replace("-GB.xlsx","-GB-export.xlsx")
print fChainesExport
fChainesCorrMentions=fChaines.replace("-GB.xlsx","-GB-corr-Mentions.html")
print fChainesCorrMentions
fChainesCorrNoMentions=fChaines.replace("-GB.xlsx","-GB-corr-NoMentions.html")
print fChainesCorrNoMentions




/Users/gilles/pCloud Drive/FOD/GB/Copy/Recherche/Rugby/ChainesReferences/FR-AR-fr-GB-corr.xlsx
/Users/gilles/pCloud Drive/FOD/GB/Copy/Recherche/Rugby/ChainesReferences/FR-AR-fr-GB-diff.xlsx
/Users/gilles/pCloud Drive/FOD/GB/Copy/Recherche/Rugby/ChainesReferences/FR-AR-fr-GB-export.xlsx
/Users/gilles/pCloud Drive/FOD/GB/Copy/Recherche/Rugby/ChainesReferences/FR-AR-fr-GB-corr-Mentions.html
/Users/gilles/pCloud Drive/FOD/GB/Copy/Recherche/Rugby/ChainesReferences/FR-AR-fr-GB-corr-NoMentions.html


In [10]:
def lireChainesXL(fNom):
    dfBrut=pd.read_excel(fNom, sheet_name="Sheet1")
    colSelection=dfBrut.columns[:6].tolist()+dfBrut.columns[11:].tolist()
    dfResult=dfBrut[colSelection]
    dfResult.columns="Modif, Referent, numChaine, chaineIndex, chaineDep, chaineApp, Transcription, Cat, Lemme".split(", ")
    dfResult=dfResult[[ \
                         "Transcription", "Cat", "Lemme", \
                         "Referent", \
                         "numChaine", "chaineIndex", "chaineDep", "chaineApp", \
                         "Modif", \
                        ]]    
    dfResult["Referent"]=dfResult.Referent.str.strip()
#     dfResult.numChaine=dfResult.numChaine.astype("Int64")
    dfResult[dfResult.numChaine.apply(isinstance, args=(float,))].numChaine=dfResult[dfResult.numChaine.apply(isinstance, args=(float,))].numChaine.astype("Int64")

    dfResult.chaineDep=dfResult.chaineDep.astype("Int64")
    dfResult.Transcription=dfResult.Transcription.astype("unicode")
#     display(dfResult)
    return dfResult
    

In [4]:
dfChaines=lireChainesXL(fChaines)
# dfChaines

# Réparations des chaînes incohérentes

### Reports et liste des chaines à référents multiples

In [11]:
def listerReferentsMultiples(df):
    
    def reportReferent(row):
        result=row
        if row["numChaine"] in dReportReferent:
            row["Referent"]=dReportReferent[row["numChaine"]]
        return row    
    
    dNumChaines=df.groupby("numChaine")["Referent"].agg(set).to_dict()
    dNumChainesPb={k:list(v) for k,v in dNumChaines.iteritems() if len(v)>1}

    dReportReferent={}
    for k,v in dNumChainesPb.iteritems():
        ref=[r for r in v if r!="REPORT"]
        if len(ref)==1:
            dReportReferent[k]=ref[0]

    df=df.apply(reportReferent,axis=1)

    for k in dReportReferent:
        dNumChainesPb.pop(k)
    lReferentsMultiples=[]
    for k,v in dNumChainesPb.iteritems():
        lReferentsMultiples.append("%05d\t"%k+", ".join(v))
    print "\n".join(sorted(lReferentsMultiples))
    return df

In [6]:
dfChaines=listerReferentsMultiples(dfChaines)

00078	JM, E
00135	J-F9, MS
00136	J-F9, MS
00138	J-F5, J-A10
00143	J-F8 + J-F6, J-F6
00175	E anglais, GTF
00192	J-A9, J-A10
00202	J-F9, J-A9
00238	IND, GTF
00247	J-F9, GPF
00264	thierry, MS
00269	J-F1 + J-A3, J-A3
00279	J-F12, J-F10 ou J-F12
00287	GTA, J-A9
00288	IND, J-A12
00291	J-A12 ou J-A13, J-F15
00304	IND, GTA
00308	IND, GTF
00320	IND, GTF
00323	IND, thierry
00330	J-F10, J-F12 + J-F13
00355	IND, GTF
00356	IND, GTA
00361	thierry, GTF
00366	IND, GPF
00385	IND, GTA
00402	J-A4 ou J-A8, J-A19
00411	IND, J-F13
00412	IND, GPA
00426	IND, J-A1
00427	IND, GTA
00432	J-F11, J-F13
00459	J-A12 ou J-A13, JML
00461	462, Buteur1
00465	J-F11, J-F15
00466	J-F11, J-A14
00482	IND, GTA
00484	IND, J-A9
00491	IND, J-F10
00492	IND, GTA
00493	GPA, GTF
00494	IND, J-F10
00497	IND, thierry
00505	IND, GTF
00509	IND, J-F2
00528	thierry, E
00529	IND, GTF
00530	IND, GTA, GTF
00537	IND, GTA
00538	GTA, GTF
00540	IND, GTF
00544	IND, GTF
00566	IND, thierry
00585	IND, GPF
00592	J-F9, J-F7
00598	J-F10, J-A14, MS
00601	

# Corrections manuelles et vérification

In [50]:
dfChainesCorr=lireChainesXL(fChainesCorr)

In [51]:
dfChainesCorr=listerReferentsMultiples(dfChainesCorr)

In [52]:
dfChainesCorr[dfChainesCorr.chaineApp.notnull()]

Transcription       Cat               Lemme Referent numChaine  \
119               le   DET:ART                  le     J-F2       NaN   
120        capitaine       NOM           capitaine     J-F2       NaN   
121               de       PRP                  de     J-F2       NaN   
122               l'   DET:ART                  le     J-F2       NaN   
123           équipe       NOM              équipe     J-F2       NaN   
124               de       PRP                  de     J-F2       NaN   
125           France       NAM              France     J-F2       NaN   
309            Mario       NAM               Mario     J-A2       NaN   
310          Ledesma       NAM             Ledesma     J-A2       NaN   
312            super       ADJ               super     J-A2       NaN   
313            Mario       NAM               Mario     J-A2       NaN   
845               l'   DET:ART                  le     MAJV       NaN   
846        excellent       ADJ           excellent     MAJV       NaN   
847     Marc-Antoine       NAM        Marc-Antoine     MAJV       NaN   
848            Jamet       NAM               Jamet     MAJV       NaN   
964           ancien       NOM              ancien      MrL       NaN   
965    international       ADJ       international      MrL       NaN   
966           ancien       ADJ              ancien      MrL       NaN   
967      trois-quart       NOM         trois-quart      MrL       NaN   
968           centre       NOM              centre      MrL       NaN   
969               de       PRP                  de      MrL       NaN   
970               l'   DET:ART                  le      MrL       NaN   
971           équipe       NOM              équipe      MrL       NaN   
972               d'       PRP                  de      MrL       NaN   
973        Argentine       NOM  argentin|argentine      MrL       NaN   
977    sélectionneur       NOM       sélectionneur      MrL       NaN   
1049           notre   DET:POS               notre       FL       NaN   
1050           homme       NOM               homme       FL       NaN   
1051              de       PRP                  de       FL       NaN   
1052         terrain       NOM             terrain       FL       NaN   
...              ...       ...                 ...      ...       ...   
10642  néo-zélandais       ADJ       néo-zélandais       TU       NaN   
11630     coéquipier       NOM          coéquipier    J-A15       NaN   
11631             du   PRP:det                  du    J-A15       NaN   
11632          Stade       NOM               stade    J-A15       NaN   
11633       Français       NAM            Français    J-A15       NaN   
11648         numéro       NOM              numéro     J-A2       NaN   
11649           deux       NUM                deux     J-A2       NaN   
11650       argentin       NOM            argentin     J-A2       NaN   
11792             l'   DET:ART                  le    J-A18       NaN   
11793          homme       NOM               homme    J-A18       NaN   
11794              a  VER:pres               avoir    J-A18       NaN   
11795     saignement       NOM          saignement    J-A18       NaN   
11812             le   DET:ART                  le   Gaëtan       NaN   
11813         centre       NOM              centre   Gaëtan       NaN   
11814             du   PRP:det                  du   Gaëtan       NaN   
11815       Biarritz       NAM            Biarritz   Gaëtan       NaN   
11816      Olympique       ADJ           olympique   Gaëtan       NaN   
15147             le   DET:ART                  le    J-F22       NaN   
15148     Toulousain       NOM          toulousain    J-F22       NaN   
15412             le   DET:ART                  le      NaN       NaN   
15413         pilier       NOM              pilier      NaN       NaN   
15414             de       PRP                  de      NaN       NaN   
15415       Clermont       NAM            Clermont      NaN      

### Liste des corrections pour l'annotateur original

In [53]:
dfComparaison=dfChainesCorr.reset_index().merge(dfChaines.reset_index(),indicator=True,how="outer")

In [54]:
dfDiff=dfComparaison[dfComparaison._merge=="left_only"]["Transcription Cat Lemme Referent numChaine chaineIndex chaineDep chaineApp Modif".split(" ")]

In [55]:
dfDiff.to_excel(fChainesDiff,encoding="utf8")

# Renumérotation des chaines

In [56]:
newNumChaines={}
iNewNumChaines=1
oldNumChaines=set()
for num in dfChainesCorr[dfChainesCorr.numChaine.notnull()].numChaine.tolist():
    if num not in oldNumChaines: 
        if isinstance(num, int):
            newNumChaines[num]=iNewNumChaines
            iNewNumChaines+=1
        else:
            iOldRefs=num.split("+")
            iOldRefs=[int(i) for i in iOldRefs]
            newStrChaine=[]
            for iOldRef in iOldRefs:
                newStrChaine.append(str(newNumChaines[iOldRef]))
            newNumChaines[num]="+".join(newStrChaine)
    oldNumChaines.add(num)

In [57]:
newNumChaines[31323]

328

In [58]:
dfChainesCorr["newNumChaine"]=dfChainesCorr["numChaine"].map(newNumChaines)
dfChainesCorr["newChaineApp"]=dfChainesCorr["chaineApp"].map(newNumChaines)

In [59]:
dfChainesCorr["Mention"]=np.NaN

In [60]:
dfTemp=dfChainesCorr["numChaine chaineIndex chaineDep chaineApp".split(" ")]

In [61]:
# dfTemp.numChaine=dfTemp.numChaine.astype("Int64")
# dfTemp[dfTemp.numChaine.notnull() & dfTemp.chaineApp.isna()]
dfTemp[dfTemp.numChaine.notnull()]

numChaine  chaineIndex  chaineDep  chaineApp
25            1          1.0        0.0        NaN
26            1          1.0        0.0        NaN
41            2          1.0        0.0        NaN
43            3          1.0        0.0        NaN
50            4          1.0        0.0        NaN
65            4          2.0        1.0        NaN
71            4          3.0        2.0        NaN
77            4          4.0        3.0        NaN
108           5          1.0        0.0        NaN
109           5          1.0        0.0        NaN
110           5          1.0        0.0        NaN
113           6          1.0        0.0        NaN
114           6          1.0        0.0        NaN
115           6          1.0        0.0        NaN
126           7          2.0        0.0        NaN
127           7          2.0        0.0        NaN
129           7          3.0        2.0        NaN
152           8          1.0        0.0        NaN
161           9          1.0        0.0        NaN
162           9          1.0        0.0        NaN
165           9          2.0        1.0        NaN
175          10          1.0        0.0        NaN
177          11          1.0        0.0        NaN
195          12          1.0        0.0        NaN
196          12          NaN        NaN        NaN
202          13          1.0        0.0        NaN
207          14          1.0        0.0        NaN
208          14          1.0        0.0        NaN
210          15          1.0        0.0        NaN
211          15          1.0        0.0        NaN
...         ...          ...        ...        ...
19783      1076          2.0        1.0        NaN
19786      1076          3.0        2.0        NaN
19824      1077          1.0        0.0        NaN
19825      1077          1.0        0.0        NaN
19826      1077          2.0        1.0        NaN
19860      1078          1.0        0.0        NaN
19862     20012          1.0        0.0        NaN
19863     20012          1.0        0.0        NaN
19864     20012          2.0        1.0        NaN
19883     20012          3.0        2.0        NaN
19894     20012          4.0        3.0        NaN
19917     20012          5.0        4.0        NaN
19923      1079          1.0        0.0        NaN
19924      1079          1.0        0.0        NaN
19938      1080          1.0        3.0        NaN
19941      1080          2.0        3.0        NaN
19943      1080          3.0        0.0        NaN
19948      1080          4.0        3.0        NaN
19960      1080          5.0        4.0        NaN
19966      1080          6.0        5.0        NaN
19995      1081          1.0        0.0        NaN
20001      1082          1.0        0.0        NaN
20003      1082          2.0        1.0        NaN
20009      1083          1.0        0.0        NaN
20023      1084          1.0        0.0        NaN
20026      1085          1.0        0.0        NaN
20032      1085          2.0        0.0        NaN
20087      1086          1.0        0.0        NaN
20088      1087          1.0        0.0        NaN
20184      1088          1.0        0.0        NaN

[2640 rows x 4 columns]

# Extraction des chaines

In [62]:
lPonct=["SENT"]
dfPonct=dfChainesCorr[dfChainesCorr["Cat"].isin(lPonct)]
indPonct=dfPonct.index.tolist()
# indPonct

In [63]:
dNumChaines=sorted(dfChainesCorr[dfChainesCorr.newNumChaine.notnull()].newNumChaine.unique().tolist())
dNumChaines

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

In [64]:
def extraitChaine(numChaine):
    bMention=False
    def marqueChaine(row):
        result=row["Transcription"]
        if row.name in indChaine:
            try:
                chaineIndex=str(int(row["chaineIndex"]))
            except:
                chaineIndex=""
            try:
                chaineDep=str(int(row["chaineDep"]))
            except:
                chaineDep=""
            result="<b><sub>%s</sub>"%chaineIndex+result+"<sub>%s</sub></b>"%chaineDep
        return result
    
    indChaine=dfChainesCorr[dfChainesCorr["newNumChaine"]==numChaine].index
    valsChaineDep=dfChainesCorr[(dfChainesCorr["newNumChaine"]==numChaine) & (dfChainesCorr["chaineDep"].notnull())]["chaineDep"].unique().tolist()
#     print valsChaineDep
    if valsChaineDep:
        valChaineDep=valsChaineDep[0]
    else:
        valChaineDep=np.NaN
    valsChaineIndex=dfChainesCorr[(dfChainesCorr["newNumChaine"]==numChaine) & (dfChainesCorr["chaineIndex"].notnull())]["chaineIndex"].unique().tolist()
    if valsChaineIndex:
        valChaineIndex=valsChaineIndex[0]
    else:
        valChaineIndex=np.NaN
    indMention=indChaine[0]
#     print indChaine.tolist(),valsChaineDep, valsChaineIndex
    if len(indChaine)==1:
        bMention=True
        dfChainesCorr.loc[indMention,"Mention"]=1
        dfChainesCorr.loc[indMention,"chaineIndex"]=1
        dfChainesCorr.loc[indMention,"chaineDep"]=0
    elif len(valsChaineDep)==1 and len(valsChaineIndex)==1:
        bMention=True
        if valChaineDep!=0: 
            print "Correction chaineDep",valChaineDep
            dfChainesCorr.loc[indMention,"chaineDep"]=0
        if valChaineIndex!=1: 
            print "chaineIndex",valChaineIndex
            dfChainesCorr.loc[indMention,"chaineIndex"]=1
        dfChainesCorr.loc[indMention,"Mention"]=1
        
    refsChaine=dfChainesCorr[dfChainesCorr["newNumChaine"]==numChaine].Referent.unique().tolist()
    
    try:
        debChaine=max([i for i in indPonct if i<indChaine.min()])
    except:
        debChaine=-1

    try:
        finChaine=min([i for i in indPonct if i>indChaine.max()])
    except:
        finChaine=dfChainesCorr.index.max()
    
    dfContexte=dfChainesCorr[(debChaine<dfChainesCorr.index) & (dfChainesCorr.index<finChaine)]
    motsContexte=dfContexte.apply(marqueChaine,axis=1).tolist()
    try:
        result=(", ".join(refsChaine)+"<br>"+" ".join(motsContexte),bMention)
    except:
        print "pb avec ', '.join"
        print numChaine
        print refsChaine
        print motsContexte
    return result


In [65]:
HTML(extraitChaine(1)[0])

In [66]:
dfChainesCorr[dfChainesCorr["newNumChaine"]==333].Referent.unique().tolist()

[u'J-F13']

In [67]:
htmlTexte=[]
htmlNoMention=[]
for k in sorted(dNumChaines[:]):
    try: 
        if k%100==0: print k,
    except:
        continue
    kLigne,bMention=extraitChaine(k)
    htmlTexte.append(kLigne)
    if not bMention:
        htmlNoMention.append(kLigne)
texteHTML="<br><br>".join(htmlTexte)
noMentionHTML="<br><br>".join(htmlNoMention)
# HTML(texteHTML)

chaineIndex 2.0
100 chaineIndex 2.0
200 Correction chaineDep 1.0
chaineIndex 2.0
300 Correction chaineDep 1.0
400 500 chaineIndex 7.0
chaineIndex 2.0
chaineIndex 2.0
600 chaineIndex 4.0
Correction chaineDep 1.0
chaineIndex 2.0
700 chaineIndex 2.0
800 chaineIndex 6.0
Correction chaineDep 2.0
900 1000 chaineIndex 2.0
1100


In [68]:
dfChainesCorr[dfChainesCorr.Mention.notnull()]

Transcription       Cat      Lemme      Referent numChaine  chaineIndex  \
25            David       NAM      David         J-F10         1          1.0   
41          Roncero       NOM    Roncero          J-A1         2          1.0   
43           Pichot       NAM     Pichot          J-A9         3          1.0   
108              le   DET:ART         le           GPF         5          1.0   
113              le   DET:ART         le           GPA         6          1.0   
152          Pichot       NAM     Pichot          J-A9         8          1.0   
175          Pichot       NAM     Pichot          J-A9        10          1.0   
177         Heymans       NAM    Heymans         J-F15        11          1.0   
195           David       NAM      David         J-F10        12          1.0   
202         Thierry       NAM    Thierry       thierry        13          1.0   
207             ces   PRO:DEM         ce           GTA        14          1.0   
210         Agustin       NAM    Agustin          J-A9        15          1.0   
260         Heymans       NAM    Heymans         J-F15        16          1.0   
307         Ledesma       NAM    Ledesma          J-A2        18          1.0   
335              l'   DET:ART         le           GPF        20          1.0   
353              du   PRP:det         du           GPF        21          1.0   
357         Ledesma  VER:cond    Ledesma          J-A2        22          1.0   
371       Hernandez  VER:pres  Hernandez         J-A10        24          1.0   
388          Cédric       NAM     Cédric         J-F15        25          1.0   
398        Monsieur       NAM   Monsieur            MS        26          1.0   
413            Juan       NAM       Juan         J-A10        27          1.0   
422          Cédric       NAM     Cédric         J-F15        28          1.0   
431              l'   DET:ART         le           GTF        29          1.0   
439              on   PRO:PER         on       IND GTF     10004          1.0   
445              on   PRO:PER         on       IND GTF     10006          1.0   
475       Hernandez       NAM  Hernandez         J-A10        31          1.0   
537            Rémy       NAM       Rémy          J-F6        33          1.0   
549        Albacete       NAM   Albacete          J-A5        34          1.0   
553       Fernandez       NAM  Fernandez  J-A4 ou J-A8        35          1.0   
565              on   PRO:PER         on       IND GTF     10008          1.0   
...             ...       ...        ...           ...       ...          ...   
19142            le   DET:ART         le          J-A1      1046          1.0   
19146            le   DET:ART         le          J-A4      1047          1.0   
19149           qui   PRO:REL        qui          1046      1048          1.0   
19177          nous   PRO:PER       nous       IND GTF     10262          1.0   
19178            on   PRO:PER         on       IND GTF     10263          1.0   
19195       Heymans       NAM    Heymans         J-F15      1050          1.0   
19224     Contepomi       NAM  Contepomi      J-A12/13      1051          1.0   
19241       Bernard       NAM    Bernard             E      1052          1.0   
19281     Contepomi       NAM  Contepomi      J-A12/13      1053          1.0   
19306       Heymans       NAM    Heymans         J-F15      1054          1.0   
19309       Jauzion       NAM    Jauzion         J-F12      1055          1.0   
19314      Dominici       NAM   Dominici         J-F11      1056          1.0   
19389            il   PRO:PER         il           IND     31058          1.0   
19396           aux   PRP:det         au           GTF      1059          1.0   
19410      Monsieur       NOM   monsieur         J-F18      1061          1.0   
19421     Elissalde       NAM  Elissalde         J-F21      1062          1.0   
19424      Bonnaire       NAM   Bonnaire         J-F19      1063          1.0   
19465            le   DET:ART        

In [69]:
outNoMentions = codecs.open(fChainesCorrNoMentions, "w",encoding="utf8")
n = outNoMentions.write(noMentionHTML)
outNoMentions.close()

In [70]:
outMentions = codecs.open(fChainesCorrMentions, "w",encoding="utf8")
n = outMentions.write(texteHTML)
outMentions.close()

In [71]:
dfChainesCorr["chaineReferent"]=np.NaN
dfExport=dfChainesCorr.copy(deep=True)
dfExport=dfExport["Transcription Cat Lemme Referent chaineReferent newNumChaine chaineIndex chaineDep newChaineApp Mention".split(" ")]
dfExport.columns="Transcription Cat Lemme Referent chaineReferent numChaine chaineIndex chaineDep chaineApp Mention".split(" ")

In [72]:
dReferents={k:0 for k in dfExport.Referent.unique().tolist() if k==k}
dReferentChaines={k:[] for k in dfExport.Referent.unique().tolist() if k==k}
print dReferents, dReferentChaines

{u'J-A4 ou J-A8': 0, u'E anglais': 0, u'J-A12 ou J-A13': 0, u'J-A13 OU J-A8': 0, u'JM': 0, u'buteurS': 0, u'J-A12/A13': 0, u'J-A22': 0, u'JB': 0, u'GP': 0, u'GPF': 0, u'GPA': 0, u'J-F7+J-F4': 0, u'Buteur1': 0, u'Marconnet': 0, u'Ga\xebtan': 0, u'buteur1': 0, u'J-F10 ou J-F12': 0, u'IND GPA': 0, u'IND GPF': 0, u'MD': 0, u'JML': 0, u'J-F20': 0, u'J-F21': 0, u'J-F22': 0, u'J-F8+J-F6': 0, u'J-F9': 0, u'J-F8': 0, u'J-F1': 0, u'ML': 0, u'J-F3': 0, u'J-F2': 0, u'J-F5': 0, u'J-F4': 0, u'J-F7': 0, u'J-F6': 0, u'MS': 0, u'olivier': 0, u'IND': 0, u'J-A19': 0, u'thierry': 0, u'FA': 0, u'FL': 0, u'J-A15': 0, u'J-A12/13': 0, u'J-A9': 0, u'J-A4': 0, u'J-A5': 0, u'J-A6': 0, u'J-A7': 0, u'J-A1': 0, u'J-A2': 0, u'J-A3': 0, u'GTA': 0, u'GTF': 0, u'GTG': 0, u'GTJ': 0, u'R': 0, u'MAJV': 0, u'IND GTF': 0, u'J-F11': 0, u'TU': 0, u'J-F14': 0, u'AA': 0, u'J-F1+J-A3': 0, u'E': 0, u'J-A12 J-A13': 0, u'J-A18': 0, u'J-F12+J-F13': 0, u'J-A16': 0, u'J-A14': 0, u'J-A4/A8': 0, u'J-A12': 0, u'J-A10': 0, u'J-A11': 0, u'

In [73]:
def numeroterChainesReferent(row):
    referent=row["Referent"]
    numChaine=row["numChaine"]
    if referent in dReferents:
        if numChaine not in dReferentChaines[referent]:
            dReferents[referent]+=1
            row["chaineReferent"]=dReferents[referent]
            dReferentChaines[referent].append(numChaine)
    return row
            
dfExport=dfExport.apply(numeroterChainesReferent,axis=1)

In [74]:
dfExport

Transcription       Cat     Lemme Referent  chaineReferent numChaine  \
0             après       PRP     après      NaN             NaN       NaN   
1             avoir  VER:infi     avoir      NaN             NaN       NaN   
2           demandé  VER:pper  demander      NaN             NaN       NaN   
3                si       KON        si      NaN             NaN       NaN   
4              tout   PRO:IND      tout      NaN             NaN       NaN   
5                le   DET:ART        le      NaN             NaN       NaN   
6             monde       NOM     monde      NaN             NaN       NaN   
7             était  VER:impf      être      NaN             NaN       NaN   
8              prêt       ADJ      prêt      NaN             NaN       NaN   
9                 ,       PUN         ,      NaN             NaN       NaN   
10               le   DET:ART        le      NaN             NaN       NaN   
11             coup       NOM      coup      NaN             NaN       NaN   
12               d'       PRP        de      NaN             NaN       NaN   
13            envoi       NOM     envoi      NaN             NaN       NaN   
14               de       PRP        de      NaN             NaN       NaN   
15               la   DET:ART        le      NaN             NaN       NaN   
16            coupe       NOM     coupe      NaN             NaN       NaN   
17               du   PRP:det        du      NaN             NaN       NaN   
18            monde       NOM     monde      NaN             NaN       NaN   
19             deux       NUM      deux      NaN             NaN       NaN   
20            mille       NUM     mille      NaN             NaN       NaN   
21             sept       NUM      sept      NaN             NaN       NaN   
22              est  VER:pres      être      NaN             NaN       NaN   
23            donné  VER:pper    donner      NaN             NaN       NaN   
24              par       PRP       par      NaN             NaN       NaN   
25            David       NAM     David    J-F10             1.0         1   
26           Skrela       NAM    Skrela    J-F10             NaN         1   
27                .      SENT         .      NaN             NaN       NaN   
28            allez  VER:pres     aller      NaN             NaN       NaN   
29               de       PRP        de      NaN             NaN       NaN   
...             ...       ...       ...      ...             ...       ...   
20160            de       PRP        de      NaN             NaN       NaN   
20161          foot       NOM      foot      NaN             NaN       NaN   
20162             ,       PUN         ,      NaN             NaN       NaN   
20163            et       KON        et      NaN             NaN       NaN   
20164          quel   PRO:REL      quel      NaN             NaN       NaN   
20165         match       NOM     match      NaN             NaN       NaN   
20166             !      SENT         !      NaN             NaN       NaN   
20167         entre       PRP     entre      NaN             NaN       NaN   
20168            l'   DET:ART        le      NaN             NaN       NaN   
20169        Italie       NAM    Italie      NaN             NaN       NaN   
20170            et       KON        et      NaN             NaN       NaN   
20171            la   DET:ART        le      NaN             NaN       NaN   
20172        France       NAM    France      NaN             NaN       NaN   
20173             .      SENT         .      NaN             NaN       NaN   
20174          très       ADV      très      NaN             NaN       NaN   
20175         bonne       ADJ       bon      NaN             NaN       NaN   
20176           fin       NOM       fin      NaN             NaN       NaN   
20177            de       PRP        de      NaN             NaN       NaN   
20178        soirée       NOM    soirée      NaN             NaN       NaN   
20179             ,       PU

In [75]:
dfExport.to_excel(fChainesExport,encoding="utf8",index=None)